# a purposeful technique for heading links

it is important for keyboard users to be able to access headings.
this is a specifically true when hygenic semantic html is written so that each
each `role=region` has a heading.

generally, links are generated from headings and inserted into the document.
i'm under the assumption there is a right way to do that so that the keyboard 
AND screen reader experience is excellent. in the wild, most implementations
consider a purely visual representation, and exclude accessibility of assistive technology.

In [1]:
%%
the list below are personal feelings about heading links:

<style>
.jp-OutputArea-output :is(dd, dt) {
    float: unset;
}
</style>
<figure>
<figcaption>personal feelings about heading links</figcaption>
 
good things about heading links
: keyboard users benefit
: partitions the document
: links to parts of the document
: easier to share links to parts of a document. 

    without heading links it requires developer literacy and work to extract an id 

bad things about heading links
: there are bad screen reader experiences
: the visual representation is arbitrary (eg. [🔗](), [¶]())
: interaction is only going for working within one browser/user-agent

</figure>

i'll be happy when the [annotation object model]/[screen reader] experience are primo,
the visual representation is functional, and the information be shared easily. ultimately,
the link will be functional in multiple contexts like reading, listening, watching and presenting.
further these features will make it easier to test cross device support.

the list below are personal feelings about heading links: 

 
 personal feelings about heading links 
 
 good things about heading links 
 
 keyboard users benefit 
 
 
 partitions the document 
 
 
 links to parts of the document 
 
 
 easier to share links to parts of a document. 
 without heading links it requires developer literacy and work to extract an id 
 
 bad things about heading links 
 
 there are bad screen reader experiences 
 
 
 the visual representation is arbitrary (eg. 🔗 , ¶ ) 
 
 
 interaction is only going for working within one browser/user-agent 
 
 
 
 i'll be happy when the [annotation object model]/[screen reader] experience are primo,
the visual representation is functional, and the information be shared easily. ultimately,
the link will be functional in multiple contexts like reading, listening, watching and presenting.
further these features will make it easier to test cross device support.

In [2]:
%%
## qr codes as heading links

the solution we'll attempt is to replace the heading link with QR codes,
and determine the best location of the link. first off, my primary reason 
for doing this is that i am a chaoctic person.
i like the entropy of QR codes and the difference of the links are distinguished visually.

### conformations of permalinks and headings

when we are messing with links we need respect [sc 2.4.4](https://www.w3.org/WAI/WCAG22/Understanding/link-purpose-in-context.html).
the links we are consdering are only in-page navigation links.

we cannot know we are inside a heading, we can only know we are at a heading

<figure><figcaption>there are four variants we can choose for associating a permalink with a heading.</figcaption>
    
link inside heading
: ```html
    <h1><a class=perma></a></h1>
  ```
: keyboard and screen reader users will have to hear the heading twice without proper consideration. this is labor increasing.

block link around heading
: ```html
    <a class=perma><h1></h1></a>
  ```
: this is the way the text should be announced, but block links are not recommended patterns
: [Block Links Are a Pain (and Maybe Just a Bad Idea)](https://css-tricks.com/block-links-are-a-pain-and-maybe-just-a-bad-idea/)
: >  [Perhaps the worst thing you can do for a block link is to wrap everything in the <a href>. ](https://adrianroselli.com/2020/02/block-links-cards-clickable-regions-etc.html?Theme=Light)
    
link before heading
: ```html
    <a class=perma></a><h1></h1>
  ```
link after heading
: ```html
    <h1></h1><a class=perma></a>
  ```

</figure>
<samp>link before heading</samp> appears to be the best solution.

        ...;
    

qr codes as heading links 
 the solution we'll attempt is to replace the heading link with QR codes,
and determine the best location of the link. first off, my primary reason
for doing this is that i am a chaoctic person.
i like the entropy of QR codes and the difference of the links are distinguished visually. 
 conformations of permalinks and headings 
 when we are messing with links we need respect sc 2.4.4 .
the links we are consdering are only in-page navigation links. 
 we cannot know we are inside a heading, we can only know we are at a heading 
 there are four variants we can choose for associating a permalink with a heading. 
 
 link inside heading 
 
 < h1 >< a class = perma ></ a ></ h1 > 
 
 
 
 keyboard and screen reader users will have to hear the heading twice without proper consideration. this is labor increasing. 
 block link around heading 
 
 < a class = perma >< h1 ></ h1 ></ a > 
 
 
 
 this is the way the text should be announced, but block links are not recommended patterns 
 Block Links Are a Pain (and Maybe Just a Bad Idea) 
 
 
 Perhaps the worst thing you can do for a block link is to wrap everything in the . 
 
 
 link before heading 
 
 < a class = perma ></ a >< h1 ></ h1 > 
 
 
 
 link after heading 
 
 < h1 ></ h1 >< a class = perma ></ a > 
 
 
 
 
 
 link before heading appears to be the best solution.
 ...;

ultimately heading links should be configurable. we'll leave them on by default
a screen reader user does not need these links and may choose to suppress them

In [3]:
%%
    module =\
```js
import QrCode from 'https://danielgjackson.github.io/qrcodejs/qrcode.mjs';
```

    module +=\
```js
var config = {"class": "anchor"}

function headingLinks() {
    console.log(headingLinks)
    document.querySelectorAll(".jp-Cell :is(h1,h2,h3,h4,h5,h6)").forEach(
        (element) => {
            let a, target, relative;
            relative = element.previousElementSibling
            if (relative?.classList.contains(config["class"])) {
                a = relative;
                a.innerHTML = ""
            } else {
                a = document.createElement("a");
                element.parentElement.insertBefore(a, element)
                a.onclick = linkHandler
            }
            a.classList.add(element.tagName.toLowerCase())
            a.classList.add(config["class"])
            if (element.tagName == "A") {
                target = element.href;
                if (!target.includes("://")) {
                    target = `${document.baseURI}#${target}`
                }
            }
            target = `${document.baseURI}#${element.id}`
            a.setAttribute("href", target)
            a.innerHTML = QrCode.render('svg', QrCode.generate(target))
            a.setAttribute("aria-labelledby", element.id)
        }
    )
}
```

    module +=\
```js
function linkHandler(event) {
    let fs;
    if (event.type == "keypress") { fs = event.key == "Enter" && event.altKey} 
    else { fs = event.altKey}
    if (fs) {event.target.requestFullscreen()}
}
```

<script type="module">
{{module}}

globalThis.headingLinks = headingLinks
headingLinks()
</script>

module =\
 
 import QrCode from 'https://danielgjackson.github.io/qrcodejs/qrcode.mjs' ; 
 
 
 module +=\
 
 var config = { "class" : "anchor" } 

 function headingLinks () { 
 console . log ( headingLinks ) 
 document . querySelectorAll ( ".jp-Cell :is(h1,h2,h3,h4,h5,h6)" ). forEach ( 
 ( element ) => { 
 let a , target , relative ; 
 relative = element . previousElementSibling 
 if ( relative ? . classList . contains ( config [ "class" ])) { 
 a = relative ; 
 a . innerHTML = "" 
 } else { 
 a = document . createElement ( "a" ); 
 element . parentElement . insertBefore ( a , element ) 
 a . onclick = linkHandler 
 } 
 a . classList . add ( element . tagName . toLowerCase ()) 
 a . classList . add ( config [ "class" ]) 
 if ( element . tagName == "A" ) { 
 target = element . href ; 
 if ( ! target . includes ( "://" )) { 
 target = ` ${ document . baseURI } # ${ target } ` 
 } 
 } 
 target = ` ${ document . baseURI } # ${ element . id } ` 
 a . setAttribute ( "href" , target ) 
 a . innerHTML = QrCode . render ( 'svg' , QrCode . generate ( target )) 
 a . setAttribute ( "aria-labelledby" , element . id ) 
 } 
 ) 
 } 
 
 
 module +=\
 
 function linkHandler ( event ) { 
 let fs ; 
 if ( event . type == "keypress" ) { fs = event . key == "Enter" && event . altKey } 
 else { fs = event . altKey } 
 if ( fs ) { event . target . requestFullscreen ()} 
 }

### interaction design

the qr codes may be visually small and indiscernible, but digital camera
zoom is not limited to that same resolution. 
qr codes are too small to use

the qr codes offer an escape from the current document and offer audiences agency
in viewing the content on their own device.

#### keyboard shortcut options

<kbd>Ctrl+Click</kbd>, <kbd>Shift+Click</kbd>, <kbd>Ctrl+Enter</kbd>, & <kbd>Shift+Enter</kbd> all have default behaviors so we choose the <kbd>Alt+Click</kbd> & <kbd>Alt+Enter</kbd> as actions for keyboard and mouse users to enlarge the qr code.

#### resizing

i may have gone too far in making the qr codes resizable. perhaps it would be fun to use in 
a presentation or a stream.

In [4]:
%%
### styling


```html
<style>
.anchor {
    --s: 1rem;
    background-repeat: no-repeat;
    box-sizing: border-box;
    float: left;
    object-fit: contain;
    svg {
        height: 100%;
        width: 100%;
        object-fit: cover;
    }
    height: var(--s);
    width: var(--s);
    color: white;
    resize: both;
    overflow: hidden;
aspect-ratio: 1 / 1;

    &.h1 {--s: 2rem;}
    &.h2 {--s: 1.875rem;}
    &.h3 {--s: 1.75rem;}
    &.h4 {--s: 1.5rem;}
}
</style>
```

'<style>\n.anchor {\n    --s: 1rem;\n    background-repeat: no-repeat;\n    box-sizing: border-box;\n    float: left;\n    object-fit: contain;\n    svg {\n        height: 100%;\n        width: 100%;\n        object-fit: cover;\n    }\n    height: var(--s);\n    width: var(--s);\n    color: white;\n    resize: both;\n    overflow: hidden;\naspect-ratio: 1 / 1;\n\n    &.h1 {--s: 2rem;}\n    &.h2 {--s: 1.875rem;}\n    &.h3 {--s: 1.75rem;}\n    &.h4 {--s: 1.5rem;}\n}\n</style>\n'

styling 
 < style > 
 . anchor { 
 --s : 1 rem ; 
 background-repeat : no-repeat ; 
 box-sizing : border-box ; 
 float : left ; 
 object-fit : contain ; 
 svg { 
 height : 100 % ; 
 width : 100 % ; 
 object-fit : cover ; 
 } 
 height : var ( --s ); 
 width : var ( --s ); 
 color : white ; 
 resize : both ; 
 overflow : hidden ; 
 aspect-ratio : 1 / 1 ; 

 & . h1 { --s : 2 rem ;} 
 & . h2 { --s : 1.875 rem ;} 
 & . h3 { --s : 1.75 rem ;} 
 & . h4 { --s : 1.5 rem ;} 
 } 
 </ style >

## conclusion

well first of all i'm happy to finally implement and see this. 
i think its fun and will try to use it. along the way i'll be 
testing and improving.

extending this technique to other elements

In [5]:
## marginalia

### what does github do?

this technique sets a precedence for including QR codes in links.
so perhaps this technique extends to other links making it easier
to transfer content across devices.
extending this approach other elements

naming the perma link. use the title of the svg or the heading

there are a few ways this is useful:

* titles can be read by other devices
* all the links can be accessed on mobile, or shared across devices.
* it provides keyboard navigation to headers, and explicit links to headers

> Text in a <title> element is not rendered as part of the graphic, but browsers usually display it as a tooltip. If an element can be described by visible text, it is recommended to reference that text with an aria-labelledby attribute rather than using the <title> element.
>
> MDN

one of the primary controls an author has for technical authoring is the heading structure.

coginitive funneling

this approach is guided by the idea that the ultimate use of content is in presentations and performances.

in general, a presentation performance on a projector or screen can NOT be accessible.
it is a small dimension of the media related to an actual presentation and general purpose accessibility requires the audience the ability to consume content through their preferred inputs. there are unknowns unknowns outside the performers control that limit the degree of accessiblility that can be implemented.

i've been working on flexible desktop/mobile interfaces for computational literature. these designs offer the ability to modify layout, presentation, and other accessibility features. this means that if content is avaiable before the presentation then the audience may absorb the presentation with their personal preferences, not the author's.

we're to use permalinks as a way to move information from the presenter to the audience. the give the audience control.

personally i want to test things on mobile device screen readers

the qr UI should make it possible to move links across devices.

links need a different treatment than headings